In [1]:
import requests
import json
from tqdm import tqdm
import time

예시코드 listing_id = '6261834'

In [2]:
url = "https://www.airbnb.co.kr/api/v2/reviews"
params = {
    'key':'d306zoyjsyarp7ifhu67rjxn52tv0t20',
    'currency':'KRW',
    'locale':'ko',
    'listing_id':'6261834',
    'role':'guest',
    '_format':'for_p3',
    '_limit':'100', # 몇개의 게시물을 보여줄 것인가(100개 이하로 설정가능)
    '_offset':'0',# 몇번째 게시물부터 시작할 것인가 0부터 시작
    '_order':'language_country'}

##### https://www.airbnb.co.kr/api/v2/reviews?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&currency=KRW&locale=ko&listing_id=6261834&role=guest&_format=for_p3&_limit=7&_offset=7&_order=language_country

In [3]:
response = requests.get(url,params=params).text

In [4]:
result = json.loads(response)

- result:{'reviews', 'metadata'}
    -  reviews:[100 items]: {'collection_tag', __'comments'__, 'created_at', __'id'__, 'language', 'localized_date', 'rating', 'response', 'reviewee', __'reviewer'__}
    - metadata:{'reviews_count', 'should_show_review_translations'}

In [5]:
for k,v in result['reviews'][0].items():
     print("|| %30s || : %s" % (k,v)) # (|| key || : value) 형태로 출력

||                 collection_tag || : None
||                       comments || : 너무 깨끗하고 좋았습니다.
||                     created_at || : 2018-10-29T07:04:45Z
||                             id || : 342585456
||                       language || : ko
||                 localized_date || : 2018년 10월
||                         rating || : 5
||                       response || : 
||                       reviewee || : {'deleted': False, 'first_name': '현아', 'host_name': '현아', 'id': 24468677, 'is_superhost': False, 'picture_url': 'https://a0.muscache.com/im/pictures/5af5aaac-5719-4708-8056-b32d76f23b44.jpg?aki_policy=profile_x_medium', 'profile_path': '/users/show/24468677'}
||                       reviewer || : {'deleted': False, 'first_name': 'Doohoon', 'host_name': 'Doohoon', 'id': 81515153, 'is_superhost': False, 'picture_url': 'https://a0.muscache.com/im/pictures/user/c9774fbd-6469-4db7-81a1-f374743fcad8.jpg?aki_policy=profile_x_medium', 'profile_path': '/users/show/81515153'}


### listing_id 이용한 리뷰데이터 크롤링(서울)

In [1]:
import requests
import json
from tqdm import tqdm
import time
import os
import glob

In [2]:
os.getcwd()

'C:\\Users\\Chankoo\\Desktop\\GitHub\\BOAZ-projects\\aribnb-review-crawler'

In [3]:
os.chdir('./../airbnb-data/') # airbnb-data 폴더로 가자

In [4]:
cwd = os.getcwd()
cwd

'C:\\Users\\Chankoo\\Desktop\\GitHub\\BOAZ-projects\\airbnb-data'

In [5]:
# 14610개 숙소 정보담은 home_jsn_14610.json 파일 불러오자
with open(os.path.join(cwd,'home_jsn_14610.json'),'r') as fp: 
    home_jsn = json.load(fp)

14610개 숙소의 모든 리뷰를 긁어오자

In [6]:
listing_ids = sorted(list(home_jsn.keys())) # 우선 home_jsn에서 숙소id가져온다

In [7]:
listing_ids[:10] # str 순으로 정렬된 listing_ids

['10003340',
 '10010172',
 '10010532',
 '1001751',
 '1001777',
 '1001811',
 '1001830',
 '1001984',
 '10022309',
 '10022842']

전역변수

In [8]:
url = "https://www.airbnb.co.kr/api/v2/reviews"
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

params = {
        'key':'d306zoyjsyarp7ifhu67rjxn52tv0t20',
        'role':'guest',
        '_format':'for_p3',
#         'currency':'KRW',
#         'locale':'ko',
#         'listing_id':
#         '_limit':'100', # 몇개의 게시물을 보여줄 것인가(100개 이하로 설정가능)
#         '_offset':'0',# 몇번째 게시물부터 시작할 것인가 0부터 시작
#         '_order':'recent'
            }

review_jsn ={} # 리뷰데이터를 json으로 저장하기위해 우선 dict 생성
review_cnt_total = 0

listing_id 이용해 숙소의 메타데이터 중 리뷰 수를 가져오는 함수 get_review_count

In [9]:
def get_review_count(listing_id,params=params,url=url,headers=headers)->int:
    params_local = params.copy()
    params_local['listing_id'] = listing_id # 리스팅 id를 해당 listing_id로 설정
    listing_id_res = requests.get(url, params=params_local, headers=headers).json() # 메타데이터 따기위한 request
    
    review_count= listing_id_res['metadata']['reviews_count'] # response하는 리뷰개수가 제한적(100개)이므로 리뷰수 따라 조정필요
    
    global review_cnt_total 
    review_cnt_total += review_count # 전역변수 review_cnt_total에 합산
    
    del params_local
    return review_count
    

listing_id와 review_count 이용해 리뷰를 review_jsn에 저장하는 함수 get_reviews

In [10]:
def get_reviews(listing_id,review_count,params=params,url=url,headers=headers):
    params_local = params.copy()
    params_local['listing_id'] = listing_id
    params_local['_limit'] = 100 # 최대 100개 리뷰 보겠다
    review_count_iter = review_count//100 # 100개단위로 리뷰개수 끊어줌 -> loop 돌아갈 횟수
    review_count_rest = review_count%100 # 100개 미만 나머지 리뷰의 수 -> loop 이후 가져올 나머지 리뷰 
    
    review_jsn[listing_id] = [] # listing_id 키로 갖는 리뷰리스트 쌓자
    
    # loop 시작
    for i in range(review_count_iter): # 리뷰개수 100개 단위로 반복
        params_local['_offset'] = 100*i

        res_100_reviews = requests.get(url, params=params_local, headers=headers).json()
        review_jsn[listing_id].extend(res_100_reviews['reviews']) # listing_id 를 key로 100개의 리뷰데이터를 쌓음
    
    # loop 끝 -> 나머지 리뷰 가져오기
    if review_count_rest >0:
        params_local['_limit'] = review_count_rest # 100개 미만 나머지 리뷰에 대해
        params_local['_offset'] = 100*review_count_iter

        res_rest_reviews = requests.get(url, params=params_local, headers=headers).json()
        review_jsn[listing_id].extend(res_rest_reviews['reviews'])
        del params_local

메인


In [11]:
params

{'_format': 'for_p3',
 'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20',
 'role': 'guest'}

In [15]:
# 전역변수 초기화
review_jsn ={} 
review_cnt_total = 0
except_cnt = 0

In [24]:
if __name__=='__main__':
    # try_idx 찾기
    try_idx = 0
    print(os.getcwd()) # './aibnb-data' 임을 확인하자
    flist = glob.glob('*_seoul.json') # seoul.json으로 끝나는 파일만 리스트로 가져옴
    if flist != []:
        try_idx = len(flist)*100
    now_try_idx = try_idx # 반복상태 체크위한 now_try_idx 변수
    
    # 반복시작
    listing_ids_loop = listing_ids[try_idx:]
    for listing_id in tqdm(listing_ids_loop):
        if (now_try_idx % 100) == 99: # now_try_idx 100개 단위로 끊어 저장
            with open(os.path.join(os.getcwd(),'review_'+str(try_idx)+'_to_'+str(now_try_idx)+'_seoul.json'),'w',encoding='utf-8') as fp:
                try:
                    json.dump(review_jsn,fp,ensure_ascii=False) # flush
                    
                except UnicodeEncodeError as e:
                    print(now_try_idx, e)
                    json.dump(review_jsn,fp,ensure_ascii=True) # retry
                finally:
                    review_jsn = {} # 초기화
                    try_idx = now_try_idx+1
            
        retry = 2
        while retry >0:
            try:
                review_count = get_review_count(listing_id)
                get_reviews(listing_id,review_count)
                break
            except Exception as e:
                except_cnt += 1
                print(e.args,e.with_traceback)
                retry -= 1
                if listing_id in review_jsn.keys():
                    del review_jsn[listing_id]
                if retry == 0:
                    print("lising_id {} not stored".format(listing_id))
                    break
                continue
            finally:
                now_try_idx += 1
    
    # 마지막 저장 dic flush
    with open(os.path.join(os.getcwd(),'review_'+str(try_idx)+'_to_'+str(now_try_idx)+'_seoul.json'),'w',encoding='utf-8') as fp:
                json.dump(review_jsn,fp,ensure_ascii=False) # flush
                review_jsn = {} # 초기화

In [20]:
lst_seoul = glob.glob('*_seoul.json') # 저장된 서울 json파일 리스트로 불러오기

In [22]:
reviews = {} # 데이터 모두 합치기
for el_seoul in lst_seoul:
    with open(el_seoul,'r',encoding='utf-8') as fp:
        tmp_dic = json.load(fp)
    reviews.update(tmp_dic)

In [23]:
len(reviews) # 14610 이면 완료

297

In [18]:
review_cnt_total

11444

In [17]:
except_cnt

3

전체 데이터 저장

In [ ]:
with open('review_seoul_final.json','w') as fp:
    json.dump(reviews,fp,ensure_ascii=False)

-------------------------------------------------
백업

In [66]:



for listing_id in tqdm(listing_ids): # 숙소id listing_id 대해
    review_jsn[listing_id] = [] # listing_id 키로 갖는 리뷰리스트 쌓자
    
    params['listing_id'] = listing_id # 리스팅 id를 해당 listing_id로 설정
    
#     retry_cnt = 2 # 최대 2번동안 listing_id에 해당하는 response 요청하겠다 
#     while retry_cnt !=0:
#         try:
    listing_id_res = requests.get(url, params=params, headers=headers).json() # 메타데이터 따기위한 request
#             break
#         except Exception as e: # response 못받은 경우 retry
#             print('retry in listing_id!!',e)
#             retry_cnt -= 1
#             time.sleep(1)
#             continue
            
    review_count= listing_id_res['metadata']['reviews_count'] # response하는 리뷰개수가 제한적(100개)이므로 리뷰수 따라 조정필요
    review_cnt_total += review_count
    
    print(review_count)
    review_count_iter = review_count//100 # 100개단위로 리뷰개수 끊어줌
    review_count_rest = review_count%100 # 100개 미만 나머지 리뷰의 수
    
    print(review_count_iter,review_count_rest,'* = ',(review_count_iter)*100 + review_count_rest)
    
    for i in range(review_count_iter): # 리뷰개수 100개 단위로 반복
        print('iter:',i)
        params['_offset'] = 100*i
#         retry_cnt = 2
        
#         while retry_cnt !=0: # 최대 2번 동안 request
#             try:
        print(params)
        res_100_reviews = requests.get(url, params=params, headers=headers).json()
        review_jsn[listing_id].extend(res_100_reviews['reviews']) # listing_id 를 key로 100개의 리뷰데이터를 쌓음
#                 break
#             except Exception as e: # response 못받은 경우 retry
#                 print('retry in review!!',e)
#                 retry_cnt -= 1
#                 time.sleep(1)
#                 continue
        
     
        
        
            
    params['_limit'] = review_count_rest # 100개 미만 나머지 리뷰에 대해
    params['_offset'] = 100*review_count_iter
#     while retry_cnt !=0: # 최대 2번 동안 request
#             try:
    
    print(params)
    
    res_rest_reviews = requests.get(url, params=params, headers=headers).json()
    review_jsn[listing_id].extend(res_rest_reviews['reviews'])
#                 break
#             except Exception as e: # response 못받은 경우 retry
#                 print('retry in review!!',e)
#                 retry_cnt -= 1
#                 time.sleep(1)
#                 continue







  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

204
2 4 * =  204
iter: 0
{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'currency': 'KRW', 'locale': 'ko', 'role': 'guest', '_format': 'for_p3', '_limit': '100', '_order': 'recent', 'listing_id': '4186309', '_offset': 0}
iter: 1
{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'currency': 'KRW', 'locale': 'ko', 'role': 'guest', '_format': 'for_p3', '_limit': '100', '_order': 'recent', 'listing_id': '4186309', '_offset': 100}
{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'currency': 'KRW', 'locale': 'ko', 'role': 'guest', '_format': 'for_p3', '_limit': 4, '_order': 'recent', 'listing_id': '4186309', '_offset': 200}








 33%|████████████████████████████                                                        | 1/3 [00:06<00:12,  6.37s/it]

200
2 0 * =  200
iter: 0
{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'currency': 'KRW', 'locale': 'ko', 'role': 'guest', '_format': 'for_p3', '_limit': 4, '_order': 'recent', 'listing_id': '8008311', '_offset': 0}
iter: 1
{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'currency': 'KRW', 'locale': 'ko', 'role': 'guest', '_format': 'for_p3', '_limit': 4, '_order': 'recent', 'listing_id': '8008311', '_offset': 100}
{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'currency': 'KRW', 'locale': 'ko', 'role': 'guest', '_format': 'for_p3', '_limit': 0, '_order': 'recent', 'listing_id': '8008311', '_offset': 200}








 67%|████████████████████████████████████████████████████████                            | 2/3 [00:09<00:04,  4.51s/it]

223
2 23 * =  223
iter: 0
{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'currency': 'KRW', 'locale': 'ko', 'role': 'guest', '_format': 'for_p3', '_limit': 0, '_order': 'recent', 'listing_id': '1043588', '_offset': 0}
iter: 1
{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'currency': 'KRW', 'locale': 'ko', 'role': 'guest', '_format': 'for_p3', '_limit': 0, '_order': 'recent', 'listing_id': '1043588', '_offset': 100}
{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'currency': 'KRW', 'locale': 'ko', 'role': 'guest', '_format': 'for_p3', '_limit': 23, '_order': 'recent', 'listing_id': '1043588', '_offset': 200}








100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:16<00:00,  5.34s/it]





